In [25]:
import cv2
import numpy as np
import os
import math
import tensorflow as tf
import idx2numpy

from matplotlib import pyplot as plt       # чтобы выводить промежуточные фото в jupyter
from tensorflow import keras
from keras.models import Sequential
from keras import optimizers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape, LSTM, BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.constraints import maxnorm

# Список всех настроечных параметров/констант
WORK_DIR = 'pass_photos'
TEMP_DIR = 'pass_temp'
# TEST_FILE = 'pass_photos/1.jpeg'
IMG_HEIGHT = 1000            # требуемый размер фото для нормализации всех изображений
IMG_WIDTH = 600              # т.к. в задачу входит прочитать только ФИО, обрезаю серию/номер чтобы не усложнять распознавание
INDENT_LEFT = 220            # обрезаем фото т.к. без него получается лучше разделить фото на куски текста
INDENT_TOP = 40              # обрезаем лишнюю часть паспорта снизу
INDENT_BOTTOM = 120          # обрезаем нижние поля
SCALE_FACTOR = 8             # во сколько раз увеличиваем вырезанные слова для дальнейшей обработки букв

In [2]:
# Функция для получения списка файлов из каталога с фотографиями (как в task_1 и task_2)
def get_files(directory: str) -> list:
    names = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            names.append(os.path.join(directory, filename))

    return names

In [3]:
# Изменение размера
def scale_image(image, scale):     # принимаем объект изображения OpenCV
    
    # получаем текущий размер, вычисляем искомый и создаем измененное изображение
    height, width = image.shape[0], image.shape[1]
    img_width = int(width * scale)
    img_height = int(height * scale)
    img = cv2.resize(image, (img_width, img_height))
    #img = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_CUBIC)
    
    return img

In [4]:
# Нормализация размеров и вырезка нужной части паспорта для обработки
def normalize_size(image):     # принимаем объект изображения OpenCV
    
    # нормализуем фото к нужному размеру
    old_height = image.shape[0]     # получаем исходную высоту
    resize_scale = IMG_HEIGHT / old_height       # считаем коэффициент масштабирования изображения до требуемого
    img = scale_image(image=image, scale=resize_scale)
    new_width = img.shape[1]      # получаем новую ширину
    
    # обрезаем паспорт до страницы с фото
    x0 = INDENT_LEFT                            # отступ слева, т.к. корочка и фото нам не важны
    y0 = IMG_HEIGHT // 2 + INDENT_TOP           # обрезка сверху, т.к. верхняя страница с местом выдачи нам не важна 
    x1 = new_width if new_width < IMG_WIDTH else IMG_WIDTH     # обрезаем все лишнее справа, если есть разворот с пропиской
    y1 = IMG_HEIGHT - INDENT_BOTTOM
    img = img[y0:y1, x0:x1]              # обресанный кусок изображения
    
    return img

In [5]:
# Подготовка изображений для распознавания текста
def normalize_color(image):         # принимаем объект изображения OpenCV
    
    # обесцвечиваем и пытаемся снизить шум с помощью размытия
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
    else:
        gray = image
    # blur = cv2.GaussianBlur(gray, (5,5), 0)         # коэффициент размытия подобран вручную
    
    
    # одно изображение используем для распознавания блоков текста. очередность преобраозвания найдена методом тыка
    kernel = np.ones((5,5), 'uint8')
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    img_block = cv2.erode(gray, kernel, iterations=1)
    #img_block = cv2.dilate(img_block, kernel, iterations=1)
    _, img_block = cv2.threshold(img_block, 0, 255, cv2.THRESH_OTSU, cv2.THRESH_BINARY_INV)
    img_block = cv2.morphologyEx(img_block, cv2.MORPH_OPEN, kernel, iterations=1)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    
    
    # Grayscale, Gaussian blur, Otsu's threshold
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Morph open to remove noise and invert image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    erosion = cv2.erode(gray, kernel, iterations = 1)
    dilation = cv2.dilate(gray, kernel, iterations = 1)
    invert = 255 - closing
    
    
    
    # Повышение контраста
    if len(image.shape) > 2:
        imghsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        imghsv[:,:,2] = [[max(pixel - 25, 0) if pixel < 190 else min(pixel + 25, 255) for pixel in row] for row in imghsv[:,:,2]]
        contrast = cv2.cvtColor(imghsv, cv2.COLOR_HSV2BGR)
        gray_contrast = cv2.cvtColor(contrast, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
        
    
    # при коэффициенте 3 - лучше распознается Васлевский, при 5 - Соколов и Юмакаева
    img_symbol = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 3, 2)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_TOZERO+cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    

    return img_block, gray      # Возвращаем контрастную картинку с разбивкой на блоки и простое ЧБ изображение

In [6]:
# Выделяем элементы текста из изображения
def search_blocks(image, limit: int, sort_by: str, sort_reverse=False):
    
    height, width = image.shape[0], image.shape[1]
    # получаем контуры больших пятен на изображении, внутри которых спрятан текст
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # output = image.copy()      #TODO - можно удалить
    
    print(f'Count of Block counoturs: {len(contours)}')
    blocks = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        # print("R", x, y, w, h, hierarchy[0][idx])
        # hierarchy[i][0]: следующий контур текущего уровня
        # hierarchy[i][1]: предыдущий контур текущего уровня
        # hierarchy[i][2]: первый вложенный элемент
        # hierarchy[i][3]: родительский элемент
        # if hierarchy[0][idx][3] == 0:               # если элемент не является самым крупным
        cv2.rectangle(image, (x, y), (x + w, y + h), (70, 0, 0), 1) # для отображаемой картинки
        if limit < h < height and limit < w < width:    # игнорируем маленькие блоки, а также блок размером с изображение
            block = image[y:y + h, x:x + w]     # вырезаем найденный блок из изображения
            # blocks.append((idx, y, h, x, w, block))
            # сохраняем габариты и изображение блока в список блоков. Загоняем в словарь, чтобы проще сортировать
            blocks.append({'idx': idx, 'y': y, 'h': h, 'x': x, 'w': w, 'block': block})
    
    # сортируем по нужному ключу: 'y' для вертикали или 'x' по горизонтали. так же можно и по индексу или размерам
    blocks.sort(key=lambda x: x.get(sort_by), reverse=sort_reverse)
    # print(blocks)
    return blocks
    # return image

In [7]:
# Режем блок изображения арифметически, если он шире одной высоты символа
def cut_blocks(image):
    height, width = image.shape[0], image.shape[1]
    C = 1.2       # просто коэффициент, рассчитанный на широкие буквы вроде Ж, М, Ш и т.д., чтобы их не делило
    if width < height*C:
        print(f'One symbol is True')
        return [image]
    else:
        print(f'One symbol is FALSE')
        result = []
        y, h, = 0, height      # высота и верхняя точке среза - всегда неизменны
        symbol_count = math.ceil(width / height)    # округляем символы до большего целого
        symbol_width = math.floor(width / symbol_count)   # округляем ширину в пикселях до меньшего целого
        
        # while image.shape[1] > image.shape[0]*C :
        for i in range(symbol_count):
            #symbol_count = round(image.shape[1] / image.shape[0])     # ширину делим на высоту ~ количество символов в блоке
            #symbol_count = math.ceil(image.shape[1] / image.shape[0])
            x = i * symbol_width
            print(f'y = {y}, h = {h}, x = {x}, symbol_width = {x+symbol_width}, width = {width}')
            result.append(image[y:h, x:x+symbol_width])
            print(f'symbol {i} is:\n{result[i]}')
            
        print(f'count of separeted symbols: {len(result)}')
        return result

In [ ]:
def emnist_model():
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='valid', input_shape=(28, 28, 1), activation='relu'))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(emnist_labels), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model

In [24]:
# Запускаем цикл по всем фото в рабочей папке
passports = get_files(WORK_DIR)
# col = 5    # количество колонок таблицы
words = []
for id_p, passport in enumerate(passports):     # идем по списку путей к изображениям
    # TODO - убрать сохранение промежутоных файлов, они используются только для визуального контроля
    temp_dir = os.path.join(TEMP_DIR, str(id_p))
    if not os.path.exists(temp_dir):
        os.mkdir(temp_dir)
        
    print(f'==== Image {id_p}.jpg =====')
    image = normalize_size(cv2.imread(passport))     # получаем кусок паспорта с ФИО
    img_blocks, img_gray = normalize_color(image=image)
    
    cv2.imwrite(f'{TEMP_DIR}/{id_p}_blocs.jpg', img_blocks)
    cv2.imwrite(f'{TEMP_DIR}/{id_p}_symbols.jpg', image)
    
    words = search_blocks(image=img_blocks, limit=15, sort_by='y')   #сортируем список сверху вниз
    # cv2.imshow('The First Word', words[0]['block'])
    # cv2.waitKey(0)
    # words.sort(key=lambda x: x.get('y'), reverse=True)
    # print(f'Count of words: {words}')
    
    # получаем все обнаруженные слова из файла, в котором читаются символы
    try:
        for id_w, word in enumerate(words[:]):    # можно забираем только первые 3 слова ФИО
            # из словаря обнаруженного блока текста забираем координаты и размер блока
            y, h, x, w = word['y'], word['h'], word['x'], word['w']
            img_word = img_gray[y:y + h, x:x + w]     # вырезаем слово по его координатам
            # img_word = image[y:y + h, x:x + w]     # вариант с повышением контраста
            img_word = scale_image(img_word, SCALE_FACTOR)        # увеличиваем изображение
            # cv2.imwrite(os.path.join(temp_dir, f'{word["idx"]}.jpg'), img_word)
            cv2.imwrite(os.path.join(temp_dir, f'{id_w}.jpg'), img_word)   #сохраняем файлы только для контроля
            
            word_blocks, word_text = normalize_color(image=img_word)
            #word_blocks, word_text = normalize_color(image=word_text)    # вариант с повышением контраста
            symbols = search_blocks(image=word_blocks, limit=80, sort_by='x')
            # print(f'Count of symbols: {len(symbols)}')
            
            for id_s, symbol in enumerate(symbols):
                # TODO - убрать сохранение промежутоных файлов, они используются только для визуального контроля
                word_dir = os.path.join(temp_dir, str(id_w))     # создаем очередную вложенную папку для котроля
                if not os.path.exists(word_dir):
                    os.mkdir(word_dir)
                
                y, h, x, w = symbol['y'], symbol['h'], symbol['x'], symbol['w']
                img_symbol = word_text[y:y + h, x:x + w]
                #cv2.imwrite(os.path.join(word_dir, f'{symbol[0]}-{e}.jpg'), img_symbol)
                
                # Доп. проверка на случай, если буквы плохо отделились
                for id_o, one_symbol in enumerate(cut_blocks(img_symbol)):
                    # print(f'element: {e}, one_symbol: {one_symbol}')
                    cv2.imwrite(os.path.join(word_dir, f'{id_s}-{id_o}.jpg'), one_symbol)
                    

            
    except IndexError:             # хотя сейчас Питон не выдает исключение, даже если в списке всего 1 элемент
        print("В паспорте найдено недостаточно данных. Попробуйте сделать более качественный скан.")
    
    
    #plt.subplot(len(passports)//col+1, col, i+1)    # высчитываем высоту таблицы от количества колонок и общего размера списка
    #plt.imshow(image, 'gray')
    # plt.title(p)
    #plt.xticks([]),plt.yticks([])
#plt.show()

==== Image 0.jpg =====
Count of Block counoturs: 18
Count of Block counoturs: 7
One symbol is True
One symbol is True
One symbol is FALSE
y = 0, h = 121, x = 0, symbol_width = 116, width = 349
symbol 0 is:
[[213 212 212 ... 204 202 200]
 [213 213 213 ... 203 201 199]
 [214 213 213 ... 202 200 198]
 ...
 [199 195 192 ... 168 167 166]
 [202 199 195 ... 174 173 172]
 [205 202 199 ... 181 180 179]]
y = 0, h = 121, x = 116, symbol_width = 232, width = 349
symbol 1 is:
[[198 196 194 ... 174 174 175]
 [197 195 193 ... 170 171 171]
 [195 193 191 ... 166 167 167]
 ...
 [165 164 163 ... 168 169 170]
 [171 171 170 ... 173 174 175]
 [178 177 176 ... 179 180 181]]
y = 0, h = 121, x = 232, symbol_width = 348, width = 349
symbol 2 is:
[[176 179 182 ... 191 192 194]
 [173 176 179 ... 188 190 192]
 [169 172 175 ... 186 188 190]
 ...
 [172 174 176 ... 189 190 192]
 [177 179 180 ... 192 194 195]
 [182 183 185 ... 196 197 198]]
count of separeted symbols: 3
Count of Block counoturs: 11
One symbol is True


Count of Block counoturs: 22
Count of Block counoturs: 19
One symbol is FALSE
y = 0, h = 119, x = 0, symbol_width = 108, width = 325
symbol 0 is:
[[158 156 155 ... 166 165 164]
 [157 156 154 ... 166 165 164]
 [157 155 153 ... 165 164 164]
 ...
 [159 157 155 ... 138 135 132]
 [161 159 157 ... 142 140 137]
 [163 161 159 ... 147 145 142]]
y = 0, h = 119, x = 108, symbol_width = 216, width = 325
symbol 1 is:
[[163 162 161 ... 167 167 167]
 [163 162 161 ... 168 168 168]
 [163 162 161 ... 169 169 169]
 ...
 [130 127 124 ... 137 137 137]
 [135 132 129 ... 141 141 141]
 [140 137 135 ... 145 145 145]]
y = 0, h = 119, x = 216, symbol_width = 324, width = 325
symbol 2 is:
[[167 167 167 ... 154 154 155]
 [168 168 168 ... 152 154 155]
 [169 169 168 ... 151 152 154]
 ...
 [137 137 138 ... 137 140 142]
 [141 142 142 ... 140 143 145]
 [146 146 146 ... 143 145 148]]
count of separeted symbols: 3
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is

Count of Block counoturs: 22
Count of Block counoturs: 13
One symbol is True
One symbol is FALSE
y = 0, h = 101, x = 0, symbol_width = 75, width = 151
symbol 0 is:
[[170 171 173 ... 167 167 166]
 [169 170 171 ... 165 165 164]
 [168 169 170 ... 163 163 162]
 ...
 [179 180 180 ... 167 166 166]
 [180 180 181 ... 169 168 168]
 [180 181 182 ... 171 170 170]]
y = 0, h = 101, x = 75, symbol_width = 150, width = 151
symbol 1 is:
[[166 166 167 ... 176 174 173]
 [164 164 164 ... 175 173 171]
 [161 161 162 ... 173 172 170]
 ...
 [165 165 166 ... 188 189 190]
 [167 167 168 ... 189 190 191]
 [169 169 170 ... 191 191 192]]
count of separeted symbols: 2
One symbol is True
One symbol is True
Count of Block counoturs: 8
One symbol is True
One symbol is FALSE
y = 0, h = 127, x = 0, symbol_width = 110, width = 221
symbol 0 is:
[[190 190 189 ... 189 190 191]
 [189 188 187 ... 187 189 190]
 [187 186 185 ... 186 187 189]
 ...
 [200 200 200 ... 183 181 178]
 [200 200 200 ... 184 182 180]
 [200 200 200 ... 18

Count of Block counoturs: 21
Count of Block counoturs: 17
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
Count of Block counoturs: 13
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
Count of Block counoturs: 14
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is True
Count of Block counoturs: 4
One symbol is True
Count of Block counoturs: 6
Count of Block counoturs: 6
One symbol is True
One symbol is True
Count of Block counoturs: 5
One symbol is True
One symbol is True
One symbol is True
Count of Block counoturs: 4
One symbol is FALSE
y = 0, h = 86, x = 0, symbol_width = 62, width = 186
symbol 0 is:
[[245 244 244 ... 241 240 240]
 [245 244 244 ... 2

Count of Block counoturs: 15
Count of Block counoturs: 16
One symbol is True
One symbol is True
One symbol is True
One symbol is True
One symbol is FALSE
y = 0, h = 124, x = 0, symbol_width = 114, width = 342
symbol 0 is:
[[200 198 196 ... 201 201 202]
 [198 196 194 ... 201 201 202]
 [196 194 191 ... 201 201 201]
 ...
 [184 182 181 ... 168 167 166]
 [186 184 183 ... 171 170 169]
 [188 187 185 ... 174 174 173]]
y = 0, h = 124, x = 114, symbol_width = 228, width = 342
symbol 1 is:
[[202 202 203 ... 205 205 205]
 [202 202 203 ... 205 205 205]
 [201 202 202 ... 205 205 204]
 ...
 [165 164 162 ... 178 179 179]
 [168 167 166 ... 182 182 183]
 [172 171 170 ... 186 187 187]]
y = 0, h = 124, x = 228, symbol_width = 342, width = 342
symbol 2 is:
[[205 205 204 ... 195 197 198]
 [204 204 204 ... 194 195 196]
 [204 204 203 ... 192 194 195]
 ...
 [180 181 182 ... 194 195 196]
 [184 185 185 ... 196 197 198]
 [188 189 189 ... 198 199 199]]
count of separeted symbols: 3
One symbol is True
One symbol is